In [3]:
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory
import sqlite3
import pandas as pd
import os 
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from textwrap import dedent
from langchain.schema import SystemMessage ,HumanMessage,AIMessage
from langchain.chains.conversation.base import ConversationChain
import json
from langchain.chat_models import ChatOpenAI
import os
# import chromadb
from langchain_chroma import Chroma
from PyPDF2 import PdfReader
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import openai
from uuid import uuid4

In [4]:
load_dotenv()
#---- Set OpenAI API key 
# Change environment variable name from "OPENAI_API_KEY" to the name given in 
# your .env file.
openai.api_key = os.environ['OPENAI_API_KEY']
# api = os.environ['OPENAI_API_KEY']
# os.environ['LANGCHAIN_TRACING_V2'] = "true"
# os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
# os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [5]:

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

In [7]:

vector_store = Chroma(
    collection_name="website",
    embedding_function=embeddings,
    persist_directory="Database/db/chroma_langchain_db",  # Where to save data locally
)

In [8]:
def ingest_file(file_path):
    # Skip non-PDF files
    if not file_path.lower().endswith('.pdf'):
        print(f"Skipping non-PDF file: {file_path}")
        return
    
    print(f"Starting to ingest file: {file_path}")
    loader = PyPDFLoader(file_path)
    loaded_documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=600, chunk_overlap=200, separators=["\n", " ", ""]
    )
    documents = text_splitter.split_documents(loaded_documents)
    uuids = [str(uuid4()) for _ in range(len(documents))]
    print(f"Adding {len(documents)} documents to the vector store")
    vector_store.add_documents(documents=documents, ids=uuids)
    print(f"Finished ingesting file: {file_path}")

In [9]:
ingest_file(r'Database\nts.pdf')

Starting to ingest file: Database\nts.pdf
Adding 39 documents to the vector store
Finished ingesting file: Database\nts.pdf


In [10]:
retrive = vector_store.similarity_search_with_relevance_scores('whats your services',k=3)
retrive

[(Document(id='431cf63f-6b64-408a-95fb-2374149ec69f', metadata={'page': 0, 'source': 'Database\\nts.pdf'}, page_content='industry\n \nprofessionals\n \nwith\n \nthe\n \nknowledge\n \nand\n \ntools\n \nthey\n \nneed\n \nto\n \nthrive,\n \nwhile\n \nalso\n \nimproving\n \nbusiness\n \nproductivity\n \nthrough\n \nthe\n \napplication\n \nof\n \nadvanced,\n \ncutting-edge\n \ntechnology\n \nincluding\n \nAI\n \nand\n \nML.\n  \nVisit\n:\n “https://www.neuraltechsoft.com/index.html”  \n \n2.2  Vision:  \nOur  vision  is  to  develop  sophisticated,  practical  analytics  that  empower  business  \nfunctions\n \nand\n \nrisk\n \nmanagement\n \nto\n \nachieve\n \ngreater\n \ncompetencies,\n \nleveraging\n \nour\n \nexcellent\n \ndomain\n \nexpertise\n \nto\n \ndeliver\n \nleading-edge\n \nadvisory\n \nand'),
  0.6816378699256085),
 (Document(id='9d60a1ee-31b6-4df0-b2b2-db6cdac75d7c', metadata={'page': 0, 'source': 'Database\\nts.pdf'}, page_content='industry\n \nprofessionals\n \nwith\n \nthe

In [19]:
retrieved_content = [doc[0].page_content for doc in retrive]

# Now you can pass `retrieved_content` to your model
print(retrieved_content)


['solutions, exceeding expectations, and fostering long-term client relationships. Visit: https://www.neuraltechsoft.com/index.html\n2. Mission and Vision\n2.1 Mission:\nOur mission is to enhance the efficiency of the financial sector by equipping industry professionals with the\nknowledge and tools they need to thrive, while also improving business productivity through the application of\nadvanced, cutting-edge technology including AI and ML. Visit: https://www.neuraltechsoft.com/index.html\n2.2 Vision:', 'customer experiences, and foster growth.Visit: https://neuraltechsoft.com/solutions_ai_ml.html\n5.5 Treasury Management:\nAt Neural TechSoft, we offer treasury management solutions to optimize cash flow, manage liquidity, and mitigate\nrisks. Our services enhance financial stability with customized strategies for cash management, investment, and\nhedging, ensuring efficient operations and long-term success.Visit: https://www.neuraltechsoft.com/tressury_management.html\n6. Key Projec

In [1]:
import shutil
persist_directory = r"Database\db\chroma_langchain_db"

# Delete the existing database directory
shutil.rmtree(persist_directory, ignore_errors=True)
print("Database cleared successfully.")

Database cleared successfully.


In [2]:
import os

# Path to the Chroma SQLite database file
sqlite_file_path = r"Database/db/chroma_langchain_db/chroma.sqlite3"

# Check if the file exists and delete it
if os.path.exists(sqlite_file_path):
    os.remove(sqlite_file_path)
    print("Chroma SQLite database deleted successfully.")
else:
    print("No such database file found.")


No such database file found.


In [3]:
from phi.agent import Agent
from phi.model.groq import Groq
groq_api_key = os.environ['GROQ_API_KEY']
from phi.vectordb.chroma import ChromaDb
from phi.agent import AgentKnowledge
from phi.embedder.openai import OpenAIEmbedder

In [15]:
embeddings = OpenAIEmbedder().get_embedding("Embed me")

In [16]:

vector_store = Chroma(
    collection_name="documents",
    embedding_function=embeddings,
    persist_directory=r"Database/db/chroma_langchain_db",  # Where to save data locally
)

In [26]:
knowledge_base = ChromaDb(collection='docuements',path=r"D:\LLAMA 3\Database\db\chroma_langchain_db\chroma.sqlite3")
knowedge_wrapper = AgentKnowledge(vector_db=knowledge_base)
# knowedge_wrapper.load(recreate=True)


In [27]:
rag_agent = Agent(model = Groq(id="llama-3.3-70b-versatile"),knowledge_base=knowedge_wrapper,search_knowledge=True,add_context=True,markdown=True)

In [28]:
rag_agent.print_response('food is stale',stream=False)

Output()

ERROR    Error searching for documents: Collection docuements does not exist.

ERROR    Error searching for documents: Collection docuements does not exist.